Import Packages

In [402]:
from nltk.corpus import wordnet #dictionnary D
from nltk.metrics import edit_distance
import PyDictionary
import pytrec_eval
import BirkBeck #Spelling error C
import _io
import numpy as np
import json
import multiprocessing as mp
import time
import re
import tqdm
import warnings
import operator

corpus = open('Data/UPWARDDAT.643', 'r')
line = 0
corpus_list = []
correct_word = []
incorrect_word = []

while line <= 10:
    read_string = corpus.readline()
    temp_string = read_string.replace('\n', '')
    string = temp_string.split("  ")
    corpus_list.append(string)
    line += 1

for i in range(len(corpus_list)):
    correct_word.append(corpus_list[i][0])
    incorrect_word.append(corpus_list[i][1])

print(correct_word)
print(incorrect_word)


['Eastwood', 'Hitler', "I'd", "I've", 'Ireland', 'Rolls', 'Scots', 'a_few', 'a_little', 'a_long [time]', 'a_lot']
['Estwood', 'Hitlar', "I'ld", 'Iev', 'Irland', 'rowles', 'Scotts', 'afew', 'alittle', 'along [time]', 'alot 26']


Minimum Edit Distance Algorithm

In [403]:
def MED(source, target):
    s = len(source)
    t = len(target)
    del_cost = 1
    ins_cost = 1
    sub_cost = 0
    distance = np.zeros((s+1, t+1))

    for i in range(1, s+1):
        distance[i,0] = i
    for j in range(1,t+1):
        distance[0,j] = j

    for i in range(1, s+1):
        for j in range(1, t+1):
            func1 = distance[i, j-1]
            func2 = distance[i-1, j]
            func3 = distance[i-1, j-1]

            if source[i-1] == target[j-1]:
                sub_cost = 0
            else:
                sub_cost = 2
            
            if func1 <= func2 and func1 <= func3:
                distance[i, j] = func1 + ins_cost
            elif func2 <= func1 and func2 <= func3:
                distance[i, j] = func2 + del_cost
            else:
                distance[i, j] = func3 + sub_cost

    return distance[s, t]

Test MED function

In [404]:
def find_nearest_token(token):
    warnings.filterwarnings('ignore') #Remove SyntaxWarning for the special_char variable
    special_char = '|!@#$%^&*()_-=+,.<>/\?;:~`1234567890'

    words = [w for w in wordnet.all_lemma_names()
        if len([c for c in list(token) if c in list(w)]) > 0 and not any (c in special_char for c in w)]

    nearest_token = sorted([(MED(token, w), w) for w in words], key=lambda n:n[0])
    return nearest_token

def convert_to_dict(list):
    ir = iter(list)
    dictionary = dict(zip(ir, ir))
    return dictionary

def find_most_simular(token, check_list):
    sorted_list = sorted(check_list.items(), key=operator.itemgetter(1))
    for i in range(len(sorted_list)):
        if (sorted_list[i][0] > 5):
            sorted_list[i] = ''
    sorted_list = list(filter(lambda a: a != '', sorted_list))
    return sorted_list

    #return min((MED(token, w), w) for w in check_list)[1]

    """"temp1 = {}
    count = 0

    for i in check_list:
        if len(token) < len(i):
            str1 = token
            str2 = i
        else:
            str1 = i
            str2 = token
        for j in range(len(str1)):
            if str1[j] == str2 [j]:
                count += 1
        temp1[i] = count
    
    temp1_value = np.array(list(temp1.values()))
    temp2 = np.argsort(temp1_value, axis=0, kind='quicksort')[-1]
    most_simular = np.array(list(temp1.keys()))[temp2]
    return most_simular, temp2"""

In [405]:
incorrect_spell = "desing"
correct_spell = "design"
checked_token = find_nearest_token(incorrect_spell)
print(checked_token)
temp_token = []
for i in range(len(checked_token)):
    temp_token.append(checked_token[i][0])
    temp_token.append(checked_token[i][1])

token = convert_to_dict(temp_token)
print(find_most_simular(incorrect_spell, token))
 

[(2.0, 'debasing'), (2.0, 'defusing'), (2.0, 'design'), (2.0, 'devising'), (2.0, 'ding'), (2.0, 'dressing'), (2.0, 'dyeing'), (2.0, 'easing'), (2.0, 'sing'), (3.0, 'dashing'), (3.0, 'deserving'), (3.0, 'designing'), (3.0, 'dingy'), (3.0, 'dying'), (3.0, 'ensuing'), (3.0, 'jesting'), (3.0, 'teasing'), (3.0, 'being'), (3.0, 'dealing'), (3.0, 'den'), (3.0, 'des'), (3.0, 'despising'), (3.0, 'destiny'), (3.0, 'dieting'), (3.0, 'dig'), (3.0, 'din'), (3.0, 'dinge'), (3.0, 'dingo'), (3.0, 'doeskin'), (3.0, 'dousing'), (3.0, 'dowsing'), (3.0, 'eosin'), (3.0, 'lessing'), (3.0, 'meshing'), (3.0, 'resin'), (3.0, 'sensing'), (3.0, 'sin'), (3.0, 'singe'), (3.0, 'sling'), (3.0, 'sting'), (3.0, 'swing'), (3.0, 'testing'), (3.0, 'using'), (3.0, 'deign'), (3.0, 'destain'), (3.0, 'destine'), (4.0, 'ageing'), (4.0, 'daring'), (4.0, 'deciding'), (4.0, 'decreasing'), (4.0, 'depressing'), (4.0, 'descending'), (4.0, 'designed'), (4.0, 'despairing'), (4.0, 'destined'), (4.0, 'doting'), (4.0, 'drowsing'), (4.0,

In [406]:
s1 = 0
s5 = 1
s10 = 1
count = 0

for i in range(len(checked_token)):
    if checked_token[i][1] == correct_spell:
        count += 1
        if checked_token[i][0] == 1:
            s1 += 1
        elif checked_token[i][0] == 5:
            s5 += 1
        elif checked_token[i][0] == 10:
            s10 =+ 1

print("k=1: ", s1)
print("k=5: ", s5)
print("k=10: ", s10)

k=1:  0
k=5:  1
k=10:  1
